## <span style='background :#3FBFBF' >*******************  IMPORTS *******************</span>  

In [1074]:
# Import
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')
#
#df_td = pd.DataFrame()

## <span style='background :#3FBFBF' >*******************  DATA SETUP *******************</span>  

In [1075]:

pd.options.display.max_columns = 999      # Allos us to display all columns
pd.options.display.max_rows = 999         # Allos us to display all columns

df_train = df_train.reindex(sorted(df_train.columns),axis=1)            # puts cols in alpabetic order
df_test = df_test.reindex(sorted(df_test.columns),axis=1)  # puts cols in alpabetic order
df_train.columns = df_train.columns.str.lower()                   # column names to lower case
df_test.columns = df_test.columns.str.lower()         # column names to lower case

## <span style='background :#3FBFBF' >*******************  FIRST LOOK *******************</span>  

### Is the shape what we expect to see? Do the number of columns match? </br>  

In [1076]:
df_train.shape, df_test.shape       
# 81 columns (features) in train, w/ 2051 rows
# 80 columns (features) in test w/ 878 rows
# Saleprice is missing from test as that is our target

((2051, 81), (878, 80))

### Examine the first few rows, and refer to the documentation that came</br>  with the data to understand the columns / ordinal / nominal data

In [1077]:
df_train.head()  

,1st flr sf,2nd flr sf,3ssn porch,alley,bedroom abvgr,bldg type,bsmt cond,bsmt exposure,bsmt full bath,bsmt half bath,bsmt qual,bsmt unf sf,bsmtfin sf 1,bsmtfin sf 2,bsmtfin type 1,bsmtfin type 2,central air,condition 1,condition 2,electrical,enclosed porch,exter cond,exter qual,exterior 1st,exterior 2nd,fence,fireplace qu,fireplaces,foundation,full bath,functional,garage area,garage cars,garage cond,garage finish,garage qual,garage type,garage yr blt,gr liv area,half bath,heating,heating qc,house style,id,kitchen abvgr,kitchen qual,land contour,land slope,lot area,lot config,lot frontage,lot shape,low qual fin sf,ms subclass,ms zoning,mas vnr area,mas vnr type,misc feature,misc val,mo sold,neighborhood,open porch sf,overall cond,overall qual,pid,paved drive,pool area,pool qc,roof matl,roof style,sale type,saleprice,screen porch,street,totrms abvgrd,total bsmt sf,utilities,wood deck sf,year built,year remod/add,yr sold
0,725,754,0,NaN,3,1Fam,TA,No,0.00,0.00,TA,192.00,533.00,0.00,GLQ,Unf,Y,RRAe,Norm,SBrkr,0,TA,Gd,HdBoard,Plywood,NaN,NaN,0,CBlock,2,Typ,475.00,2.00,TA,RFn,TA,Attchd,1976.00,1479,1,GasA,Ex,2Story,109,1,Gd,Lvl,Gtl,13517,CulDSac,NaN,IR1,0,60,RL,289.00,BrkFace,NaN,0,3,Sawyer,44,8,6,533352170,Y,0,NaN,CompShg,Gable,WD,130500,0,Pave,6,725.00,AllPub,0,1976,2005,2010
1,913,1209,0,NaN,4,1Fam,TA,No,1.00,0.00,Gd,276.00,637.00,0.00,GLQ,Unf,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,559.00,2.00,TA,RFn,TA,Attchd,1997.00,2122,1,GasA,Ex,2Story,544,1,Gd,Lvl,Gtl,11492,CulDSac,43.00,IR1,0,60,RL,132.00,BrkFace,NaN,0,4,SawyerW,74,5,7,531379050,Y,0,NaN,CompShg,Gable,WD,220000,0,Pave,8,913.00,AllPub,0,1996,1997,2009
2,1057,0,0,NaN,3,1Fam,TA,No,1.00,0.00,TA,326.00,731.00,0.00,GLQ,Unf,Y,Norm,Norm,SBrkr,0,Gd,TA,VinylSd,VinylSd,NaN,NaN,0,CBlock,1,Typ,246.00,1.00,TA,Unf,TA,Detchd,1953.00,1057,0,GasA,TA,1Story,153,1,Gd,Lvl,Gtl,7922,Inside,68.00,Reg,0,20,RL,0.00,None,NaN,0,1,NAmes,52,7,5,535304180,Y,0,NaN,CompShg,Gable,WD,109000,0,Pave,5,1057.00,AllPub,0,1953,2007,2010
3,744,700,0,NaN,3,1Fam,TA,No,0.00,0.00,Gd,384.00,0.00,0.00,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,400.00,2.00,TA,Fin,TA,BuiltIn,2007.00,1444,1,GasA,Gd,2Story,318,1,TA,Lvl,Gtl,9802,Inside,73.00,Reg,0,60,RL,0.00,None,NaN,0,4,Timber,0,5,5,916386060,Y,0,NaN,CompShg,Gable,WD,174000,0,Pave,7,384.00,AllPub,100,2006,2007,2010
4,831,614,0,NaN,3,1Fam,Gd,No,0.00,0.00,Fa,676.00,0.00,0.00,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,Wd Sdng,Plywood,NaN,NaN,0,PConc,2,Typ,484.00,2.00,TA,Unf,TA,Detchd,1957.00,1445,0,GasA,TA,1.5Fin,255,1,TA,Lvl,Gtl,14235,Inside,82.00,IR1,0,50,RL,0.00,None,NaN,0,3,SawyerW,59,8,6,906425045,N,0,NaN,CompShg,Gable,WD,138500,0,Pave,6,676.00,AllPub,0,1900,1993,2010


In [1078]:
df_test.head() 

,1st flr sf,2nd flr sf,3ssn porch,alley,bedroom abvgr,bldg type,bsmt cond,bsmt exposure,bsmt full bath,bsmt half bath,bsmt qual,bsmt unf sf,bsmtfin sf 1,bsmtfin sf 2,bsmtfin type 1,bsmtfin type 2,central air,condition 1,condition 2,electrical,enclosed porch,exter cond,exter qual,exterior 1st,exterior 2nd,fence,fireplace qu,fireplaces,foundation,full bath,functional,garage area,garage cars,garage cond,garage finish,garage qual,garage type,garage yr blt,gr liv area,half bath,heating,heating qc,house style,id,kitchen abvgr,kitchen qual,land contour,land slope,lot area,lot config,lot frontage,lot shape,low qual fin sf,ms subclass,ms zoning,mas vnr area,mas vnr type,misc feature,misc val,mo sold,neighborhood,open porch sf,overall cond,overall qual,pid,paved drive,pool area,pool qc,roof matl,roof style,sale type,screen porch,street,totrms abvgrd,total bsmt sf,utilities,wood deck sf,year built,year remod/add,yr sold
0,908,1020,0,Grvl,4,2fmCon,TA,No,0,0,Fa,1020,0,0,Unf,Unf,N,Norm,Norm,FuseP,112,Fa,TA,AsbShng,AsbShng,NaN,NaN,0,Stone,2,Typ,440,1,Po,Unf,Po,Detchd,1910.00,1928,0,GasA,Gd,2Story,2658,2,Fa,Lvl,Gtl,9142,Inside,69.00,Reg,0,190,RM,0.00,None,NaN,0,4,OldTown,60,8,6,902301120,Y,0,NaN,CompShg,Gable,WD,0,Pave,9,1020,AllPub,0,1910,1950,2006
1,1967,0,0,NaN,6,Duplex,TA,No,0,0,Gd,1967,0,0,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,Plywood,Plywood,NaN,NaN,0,CBlock,2,Typ,580,2,TA,Fin,TA,Attchd,1977.00,1967,0,GasA,TA,1Story,2718,2,TA,Lvl,Gtl,9662,Inside,NaN,IR1,0,90,RL,0.00,None,NaN,0,8,Sawyer,0,4,5,905108090,Y,0,NaN,CompShg,Gable,WD,0,Pave,10,1967,AllPub,170,1977,1977,2006
2,664,832,0,NaN,3,1Fam,Gd,Av,1,0,Gd,100,554,0,GLQ,Unf,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,Gd,1,PConc,2,Typ,426,2,TA,RFn,TA,Attchd,2006.00,1496,1,GasA,Ex,2Story,2414,1,Gd,Lvl,Gtl,17104,Inside,58.00,IR1,0,60,RL,0.00,None,NaN,0,9,Gilbert,24,5,7,528218130,Y,0,NaN,CompShg,Gable,New,0,Pave,7,654,AllPub,100,2006,2006,2006
3,968,0,0,NaN,2,1Fam,TA,No,0,0,TA,968,0,0,Unf,Unf,Y,Norm,Norm,SBrkr,184,TA,Gd,Wd Sdng,Wd Sdng,NaN,NaN,0,CBlock,1,Typ,480,2,TA,Unf,Fa,Detchd,1935.00,968,0,GasA,TA,1Story,1989,1,TA,Lvl,Gtl,8520,Inside,60.00,Reg,0,30,RM,0.00,None,NaN,0,7,OldTown,0,6,5,902207150,N,0,NaN,CompShg,Gable,WD,0,Pave,5,968,AllPub,0,1923,2006,2007
4,1394,0,0,NaN,3,1Fam,TA,No,1,0,Gd,785,609,0,BLQ,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,Plywood,Plywood,NaN,Gd,2,CBlock,1,Typ,514,2,TA,RFn,TA,Attchd,1963.00,1394,1,GasA,Gd,1Story,625,1,TA,Lvl,Gtl,9500,Inside,NaN,IR1,0,20,RL,247.00,BrkFace,NaN,0,7,NAmes,76,5,6,535105100,Y,0,NaN,CompShg,Gable,WD,185,Pave,6,1394,AllPub,0,1963,1963,2009


### Check for nulls and data types </br>  

In [1079]:
df_train.info()      # A bunhc of nulls that we needs to clean up, a bunch of object types we may need to deal with

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   1st flr sf       2051 non-null   int64  
 1   2nd flr sf       2051 non-null   int64  
 2   3ssn porch       2051 non-null   int64  
 3   alley            140 non-null    object 
 4   bedroom abvgr    2051 non-null   int64  
 5   bldg type        2051 non-null   object 
 6   bsmt cond        1996 non-null   object 
 7   bsmt exposure    1993 non-null   object 
 8   bsmt full bath   2049 non-null   float64
 9   bsmt half bath   2049 non-null   float64
 10  bsmt qual        1996 non-null   object 
 11  bsmt unf sf      2050 non-null   float64
 12  bsmtfin sf 1     2050 non-null   float64
 13  bsmtfin sf 2     2050 non-null   float64
 14  bsmtfin type 1   1996 non-null   object 
 15  bsmtfin type 2   1995 non-null   object 
 16  central air      2051 non-null   object 
 17  condition 1   

In [1080]:
df_test.info()      # A bunhc of nulls that we needs to clean up, a bunch of object types we may need to deal with

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   1st flr sf       878 non-null    int64  
 1   2nd flr sf       878 non-null    int64  
 2   3ssn porch       878 non-null    int64  
 3   alley            58 non-null     object 
 4   bedroom abvgr    878 non-null    int64  
 5   bldg type        878 non-null    object 
 6   bsmt cond        853 non-null    object 
 7   bsmt exposure    853 non-null    object 
 8   bsmt full bath   878 non-null    int64  
 9   bsmt half bath   878 non-null    int64  
 10  bsmt qual        853 non-null    object 
 11  bsmt unf sf      878 non-null    int64  
 12  bsmtfin sf 1     878 non-null    int64  
 13  bsmtfin sf 2     878 non-null    int64  
 14  bsmtfin type 1   853 non-null    object 
 15  bsmtfin type 2   853 non-null    object 
 16  central air      878 non-null    object 
 17  condition 1     

# Anything odd in the numeric data?

In [1081]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)   # Editing the format so we can better look at the description data
df_train.describe(include=[np.number]).T                            # Use .T to transpose and make it easier to read

# with help from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
# and https://stackoverflow.com/questions/55394854/how-to-change-the-format-of-describe-output

# I don't see any negative min numbers
# Looking at the difference between Min 25% - 75% and Max for outliers investigate later...
# 1st flr sf  
# bedroom abvgr
# bsmt full bath
# bsmt unf sf
# bsmtfin sf 1
# bsmtfin sf 2 - what is this?
# fireplaces
# garage area
# gr liv area
# lot area
# lot frontage
# low qual fin sf - weird?
# mas vnr area - weird?
# open porch sf
# saleprice
# totrms abvgrd
# wood deck sf

,count,mean,std,min,25%,50%,75%,max
1st flr sf,2051.00,1164.49,396.45,334.00,879.50,1093.00,1405.00,5095.00
2nd flr sf,2051.00,329.33,425.67,0.00,0.00,0.00,692.50,1862.00
3ssn porch,2051.00,2.59,25.23,0.00,0.00,0.00,0.00,508.00
bedroom abvgr,2051.00,2.84,0.83,0.00,2.00,3.00,3.00,8.00
bsmt full bath,2049.00,0.43,0.52,0.00,0.00,0.00,1.00,3.00
bsmt half bath,2049.00,0.06,0.25,0.00,0.00,0.00,0.00,2.00
bsmt unf sf,2050.00,567.73,444.95,0.00,220.00,474.50,811.00,2336.00
bsmtfin sf 1,2050.00,442.30,461.20,0.00,0.00,368.00,733.75,5644.00
bsmtfin sf 2,2050.00,47.96,165.00,0.00,0.00,0.00,0.00,1474.00
enclosed porch,2051.00,22.57,59.85,0.00,0.00,0.00,0.00,432.00


In [1082]:
df_test.describe(include=[np.number]).T 

,count,mean,std,min,25%,50%,75%,max
1st flr sf,878.00,1145.79,375.26,407.00,864.00,1063.00,1358.00,2674.00
2nd flr sf,878.00,348.40,432.99,0.00,0.00,0.00,720.00,2065.00
3ssn porch,878.00,2.60,24.96,0.00,0.00,0.00,0.00,360.00
bedroom abvgr,878.00,2.88,0.83,0.00,2.00,3.00,3.00,6.00
bsmt full bath,878.00,0.44,0.53,0.00,0.00,0.00,1.00,2.00
bsmt half bath,878.00,0.06,0.23,0.00,0.00,0.00,0.00,1.00
bsmt unf sf,878.00,539.13,426.17,0.00,216.00,452.00,780.00,2046.00
bsmtfin sf 1,878.00,441.33,438.46,0.00,0.00,373.00,734.75,2288.00
bsmtfin sf 2,878.00,53.90,178.63,0.00,0.00,0.00,0.00,1526.00
enclosed porch,878.00,24.06,73.25,0.00,0.00,0.00,0.00,1012.00


### What does the object (string) data look like?

Features with 2 unique numbers we could binarize
Features with fewer than 5 or 6 maybe ordinals that we could convert, confirm with the documentation
Do the number of unique values match for the test and train data?

In [1083]:
df_train.describe(include=[object])  

,alley,bldg type,bsmt cond,bsmt exposure,bsmt qual,bsmtfin type 1,bsmtfin type 2,central air,condition 1,condition 2,electrical,exter cond,exter qual,exterior 1st,exterior 2nd,fence,fireplace qu,foundation,functional,garage cond,garage finish,garage qual,garage type,heating,heating qc,house style,kitchen qual,land contour,land slope,lot config,lot shape,ms zoning,mas vnr type,misc feature,neighborhood,paved drive,pool qc,roof matl,roof style,sale type,street,utilities
count,140,2051,1996,1993,1996,1996,1995,2051,2051,2051,2051,2051,2051,2051,2051,400,1051,2051,2051,1937,1937,1937,1938,2051,2051,2051,2051,2051,2051,2051,2051,2051,2029,65,2051,2051,9,2051,2051,2051,2051,2051
unique,2,5,5,4,5,6,6,2,9,8,5,5,4,15,15,4,5,6,8,5,3,5,6,5,5,8,4,4,3,5,4,7,4,5,28,3,4,6,6,9,2,3
top,Grvl,1Fam,TA,No,TA,GLQ,Unf,Y,Norm,Norm,SBrkr,TA,TA,VinylSd,VinylSd,MnPrv,Gd,PConc,Typ,TA,Unf,TA,Attchd,GasA,Ex,1Story,TA,Lvl,Gtl,Inside,Reg,RL,None,Shed,NAmes,Y,Gd,CompShg,Gable,WD,Pave,AllPub
freq,85,1700,1834,1339,887,615,1749,1910,1767,2025,1868,1778,1247,724,721,227,523,926,1915,1868,849,1832,1213,2018,1065,1059,1047,1843,1953,1503,1295,1598,1218,56,310,1861,4,2025,1619,1781,2044,2049


In [1084]:
df_test.describe(include=[object])  

,alley,bldg type,bsmt cond,bsmt exposure,bsmt qual,bsmtfin type 1,bsmtfin type 2,central air,condition 1,condition 2,electrical,exter cond,exter qual,exterior 1st,exterior 2nd,fence,fireplace qu,foundation,functional,garage cond,garage finish,garage qual,garage type,heating,heating qc,house style,kitchen qual,land contour,land slope,lot config,lot shape,ms zoning,mas vnr type,misc feature,neighborhood,paved drive,pool qc,roof matl,roof style,sale type,street,utilities
count,58,878,853,853,853,853,853,878,878,878,877,878,878,878,878,172,456,878,878,833,833,833,834,878,878,878,878,878,878,878,878,878,877,41,878,878,4,878,878,878,878,878
unique,2,5,3,4,5,6,6,2,9,3,4,5,4,13,16,4,5,6,6,5,3,4,6,4,4,8,5,4,3,5,4,6,5,3,26,3,2,6,6,10,2,2
top,Grvl,1Fam,TA,No,TA,Unf,Unf,Y,Norm,Norm,SBrkr,TA,TA,VinylSd,VinylSd,MnPrv,Gd,PConc,Typ,TA,Unf,TA,Attchd,GasA,Ex,1Story,TA,Lvl,Gtl,Inside,Reg,RL,None,Shed,NAmes,Y,Ex,CompShg,Gable,WD,Pave,AllPub
freq,35,724,781,567,396,248,749,823,755,875,813,770,552,302,294,103,220,383,812,796,382,782,518,866,429,422,447,790,835,636,564,674,534,39,133,790,3,861,702,755,873,877


In [1085]:
# Print a list of columns with 2 or 3 values so we can check the documentation 
# We're include 3 unique values just in case 'Null' or 'NA' or something to that 
# affect is being picked up as a value rather than a null
# We will also check the documentation for a complete list, this was more an exercise 
# to test the ability incase we are missing documentation in a future scenario

for col in df_train:
    n = df_train[col].nunique()
    if n <= 3: 
        print(f'df_train{[col]} : {n} unique vals.')

# with help from https://www.geeksforgeeks.org/how-to-count-distinct-values-of-a-pandas-dataframe-column/

df_train['alley'] : 2 unique vals.
df_train['bsmt half bath'] : 3 unique vals.
df_train['central air'] : 2 unique vals.
df_train['garage finish'] : 3 unique vals.
df_train['half bath'] : 3 unique vals.
df_train['land slope'] : 3 unique vals.
df_train['paved drive'] : 3 unique vals.
df_train['street'] : 2 unique vals.
df_train['utilities'] : 3 unique vals.



## <span style='background :#3FBFBF' >*******************  FIX NaN's *******************</span>  


We will look at the train data first, then the test data. Investigate the null values to determine what, if anything, should/could be there instead.

In [1086]:
df_train.isnull().sum()   
# Look at the columns with fewer Nan's first. 
# A lot of the basement features are NaN in similar amounts.  
# The same for garages.

1st flr sf            0
2nd flr sf            0
3ssn porch            0
alley              1911
bedroom abvgr         0
bldg type             0
bsmt cond            55
bsmt exposure        58
bsmt full bath        2
bsmt half bath        2
bsmt qual            55
bsmt unf sf           1
bsmtfin sf 1          1
bsmtfin sf 2          1
bsmtfin type 1       55
bsmtfin type 2       56
central air           0
condition 1           0
condition 2           0
electrical            0
enclosed porch        0
exter cond            0
exter qual            0
exterior 1st          0
exterior 2nd          0
fence              1651
fireplace qu       1000
fireplaces            0
foundation            0
full bath             0
functional            0
garage area           1
garage cars           1
garage cond         114
garage finish       114
garage qual         114
garage type         113
garage yr blt       114
gr liv area           0
half bath             0
heating               0
heating qc      

In [1087]:
df_test.isnull().sum()   

1st flr sf           0
2nd flr sf           0
3ssn porch           0
alley              820
bedroom abvgr        0
bldg type            0
bsmt cond           25
bsmt exposure       25
bsmt full bath       0
bsmt half bath       0
bsmt qual           25
bsmt unf sf          0
bsmtfin sf 1         0
bsmtfin sf 2         0
bsmtfin type 1      25
bsmtfin type 2      25
central air          0
condition 1          0
condition 2          0
electrical           1
enclosed porch       0
exter cond           0
exter qual           0
exterior 1st         0
exterior 2nd         0
fence              706
fireplace qu       422
fireplaces           0
foundation           0
full bath            0
functional           0
garage area          0
garage cars          0
garage cond         45
garage finish       45
garage qual         45
garage type         44
garage yr blt       45
gr liv area          0
half bath            0
heating              0
heating qc           0
house style          0
id         

## The above lists are difficult to read and make</br> comparisons between the test and train data

To fix that, we will generate a list of only columns with null values.  We will check each column in both train and test. 
Then stack them together so we can quickly identifiy which columns we need ot fix.

In [1088]:
for col in df_train:
    x = df_train[col].isnull().sum()
    if x > 0: 
        print(f'df_train{[col]} : {x} null vals.')
    if col != 'saleprice':   # Skip 'saleprice' as it is missing from the test data
        y = df_test[col].isnull().sum() 
        if y > 0: 
            print(f'df_test{[col]} : {y} null vals.')

df_train['alley'] : 1911 null vals.
df_test['alley'] : 820 null vals.
df_train['bsmt cond'] : 55 null vals.
df_test['bsmt cond'] : 25 null vals.
df_train['bsmt exposure'] : 58 null vals.
df_test['bsmt exposure'] : 25 null vals.
df_train['bsmt full bath'] : 2 null vals.
df_train['bsmt half bath'] : 2 null vals.
df_train['bsmt qual'] : 55 null vals.
df_test['bsmt qual'] : 25 null vals.
df_train['bsmt unf sf'] : 1 null vals.
df_train['bsmtfin sf 1'] : 1 null vals.
df_train['bsmtfin sf 2'] : 1 null vals.
df_train['bsmtfin type 1'] : 55 null vals.
df_test['bsmtfin type 1'] : 25 null vals.
df_train['bsmtfin type 2'] : 56 null vals.
df_test['bsmtfin type 2'] : 25 null vals.
df_test['electrical'] : 1 null vals.
df_train['fence'] : 1651 null vals.
df_test['fence'] : 706 null vals.
df_train['fireplace qu'] : 1000 null vals.
df_test['fireplace qu'] : 422 null vals.
df_train['garage area'] : 1 null vals.
df_train['garage cars'] : 1 null vals.
df_train['garage cond'] : 114 null vals.
df_test['garag

### Now we can clearly see that almost every column in </br>train with a null value also has null values in test
### When we clean/replace the nulls in one data set, </br>we can do the other at the same time.

### EXCEPTIONS

df_train['bsmt full bath'] : 2 null vals.</br>
df_train['bsmt half bath'] : 2 null vals.</br>
df_train['bsmt unf sf'] : 1 null vals.</br>
df_train['bsmtfin sf 1'] : 1 null vals.</br>
df_train['bsmtfin sf 2'] : 1 null vals.</br>
df_test['electrical'] : 1 null vals.</br>
df_train['garage area'] : 1 null vals.</br>
df_train['garage cars'] : 1 null vals.</br>
df_train['total bsmt sf'] : 1 null vals.

## We begin the investigation and clean up in alphabetic order

We could probably assume that all missing object (string) values should be NA, but really should check each one in the documentation to be safe.  While we're looking at each feature, we can make sure that the values in the Test and Train data align with each other, as well as the values mentioned in the documentation, and fix anything that does line up.

For the columns without any null values, we will perform a test below to compare they values of the Test data to those in the Train data before converting to numeric data types.

In [1089]:
def fill_nulls_convert(col,dict_fix):
    
    df_train[col] = df_train[col].fillna('NA')      # Fill in the NaN train data values
    df_test[col] = df_test[col].fillna('NA')        # Fill in the NaN test data values
    
    df_train[col] = df_train[col].replace(dict_fix)   # Replace the values in the column with the 
    df_test[col] = df_test[col].replace(dict_fix)     # appropriate valuesfrom the dictionary
    
    # with help from https://pbpython.com/categorical-encoding.html

### "alley"
According to documentation, this is a nominal feature with 2 possible values.  That lines up with what we see in the data.  We will fix the null values and convert the object values to numeric.

In [1090]:
df_train['alley'].value_counts() , df_test['alley'].value_counts()

(Grvl    85
 Pave    55
 Name: alley, dtype: int64,
 Grvl    35
 Pave    23
 Name: alley, dtype: int64)

In [1091]:
col_fix_dict = {'Grvl':2,'Pave':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('alley', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['alley'].value_counts() , df_test['alley'].value_counts()     # Double check results

(0    1911
 2      85
 1      55
 Name: alley, dtype: int64,
 0    820
 2     35
 1     23
 Name: alley, dtype: int64)

### "bsmt cond" 
According to documentation, "bsmt cond" is an ordinal feature with 7 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1092]:
df_train['bsmt cond'].value_counts() , df_test['bsmt cond'].value_counts()

(TA    1834
 Gd      89
 Fa      65
 Po       5
 Ex       3
 Name: bsmt cond, dtype: int64,
 TA    781
 Fa     39
 Gd     33
 Name: bsmt cond, dtype: int64)

In [1093]:
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt cond', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt cond'].value_counts() , df_test['bsmt cond'].value_counts()     # Double check results

(3    1834
 4      89
 2      65
 0      55
 1       5
 5       3
 Name: bsmt cond, dtype: int64,
 3    781
 2     39
 4     33
 0     25
 Name: bsmt cond, dtype: int64)

### "bsmt exposure" 
According to documentation, "bsmt exposure" is an ordinal feature with 5 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1094]:
df_train['bsmt exposure'].value_counts() , df_test['bsmt exposure'].value_counts()

(No    1339
 Av     288
 Gd     203
 Mn     163
 Name: bsmt exposure, dtype: int64,
 No    567
 Av    130
 Gd     80
 Mn     76
 Name: bsmt exposure, dtype: int64)

In [1095]:
col_fix_dict = {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt exposure', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt exposure'].value_counts() , df_test['bsmt exposure'].value_counts()     # Double check results

(1    1339
 3     288
 4     203
 2     163
 0      58
 Name: bsmt exposure, dtype: int64,
 1    567
 3    130
 4     80
 2     76
 0     25
 Name: bsmt exposure, dtype: int64)

### "bsmt full bath" 
According to documentation, "bsmt full bath" is a discrete feature representing a count of full bathrooms in the basement.  That lines up with what we see in the data, though some values are missing from the Test data.  There is no need to convert the dataanything except for the NaN's.

In [1096]:
df_train['bsmt full bath'].value_counts() , df_test['bsmt full bath'].value_counts()

(0.00    1200
 1.00     824
 2.00      23
 3.00       2
 Name: bsmt full bath, dtype: int64,
 0    507
 1    356
 2     15
 Name: bsmt full bath, dtype: int64)

In [1097]:
col_fix_dict = {'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt full bath', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt full bath'].value_counts() , df_test['bsmt full bath'].value_counts()     # Double check results

(0.00    1202
 1.00     824
 2.00      23
 3.00       2
 Name: bsmt full bath, dtype: int64,
 0    507
 1    356
 2     15
 Name: bsmt full bath, dtype: int64)

### "bsmt half bath" 
According to documentation, "bsmt halfl bath" is a discrete feature representing a count of half bathrooms in the basement.  That lines up with what we see in the data, though some values are missing from the Test data.  There is no need to convert the dataanything except for the NaN's.

In [1098]:
df_train['bsmt half bath'].value_counts() , df_test['bsmt half bath'].value_counts()

(0.00    1923
 1.00     122
 2.00       4
 Name: bsmt half bath, dtype: int64,
 0    829
 1     49
 Name: bsmt half bath, dtype: int64)

In [1099]:
col_fix_dict = {'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt half bath', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt half bath'].value_counts() , df_test['bsmt half bath'].value_counts()     # Double check results

(0.00    1925
 1.00     122
 2.00       4
 Name: bsmt half bath, dtype: int64,
 0    829
 1     49
 Name: bsmt half bath, dtype: int64)

### "bsmt qual" 
According to documentation, "bsmt qual" is an ordinal feature with 6 possible values.  That lines up with what we see in the data. We will fix the null values and convert the object values to numeric.

In [1100]:
df_train['bsmt qual'].value_counts() , df_test['bsmt qual'].value_counts()

(TA    887
 Gd    864
 Ex    184
 Fa     60
 Po      1
 Name: bsmt qual, dtype: int64,
 TA    396
 Gd    355
 Ex     73
 Fa     28
 Po      1
 Name: bsmt qual, dtype: int64)

In [1101]:
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt qual', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt qual'].value_counts() , df_test['bsmt qual'].value_counts()     # Double check results

(3    887
 4    864
 5    184
 2     60
 0     55
 1      1
 Name: bsmt qual, dtype: int64,
 3    396
 4    355
 5     73
 2     28
 0     25
 1      1
 Name: bsmt qual, dtype: int64)

### "bsmt unf sf" 
According to documentation, "bsmt unf sf" is a continuous feature representing unfinished square feet.  We expect many unique values. We will fix the null values and convert the object values to numeric.

In [1102]:
df_train['bsmt unf sf'].isnull().sum(), df_test['bsmt unf sf'].isnull().sum()

(1, 0)

In [1103]:
col_fix_dict = {'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmt unf sf', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmt unf sf'].isnull().sum() , df_test['bsmt unf sf'].isnull().sum()     # Double check results

(0, 0)

### "bsmtfin sf 1" 
According to documentation, "bsmtfin sf 1" is a continuous feature representing finished square feet.  We expect many unique values. We will fix the null values and convert the object values to numeric.

In [1104]:
df_train['bsmtfin sf 1'].isnull().sum(), df_test['bsmtfin sf 1'].isnull().sum()

(1, 0)

In [1105]:
col_fix_dict = {'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert('bsmtfin sf 1', col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train['bsmtfin sf 1'].isnull().sum() , df_test['bsmtfin sf 1'].isnull().sum()     # Double check results

(0, 0)

### "bsmtfin sf 2" 
According to documentation, "bsmtfin sf 2" is a continuous feature representing finished square feet.  We expect many unique values. We will fix the null values and convert the object values to numeric.

In [1106]:
df_train['bsmtfin sf 2'].isnull().sum(), df_test['bsmtfin sf 2'].isnull().sum()

(1, 0)

In [1107]:
col_name = "bsmtfin sf 2"
col_fix_dict = {'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].isnull().sum() , df_test[col_name].isnull().sum()     # Double check results

(0, 0)

### "bsmtfin type 1" 
According to documentation, "bsmt cond" is an ordinal feature with 7 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1108]:
df_train['bsmtfin type 1'].value_counts() , df_test['bsmtfin type 1'].value_counts()

(GLQ    615
 Unf    603
 ALQ    293
 BLQ    200
 Rec    183
 LwQ    102
 Name: bsmtfin type 1, dtype: int64,
 Unf    248
 GLQ    243
 ALQ    136
 Rec    105
 BLQ     69
 LwQ     52
 Name: bsmtfin type 1, dtype: int64)

In [1109]:
col_name = "bsmtfin type 1"
col_fix_dict = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(6    615
 1    603
 5    293
 4    200
 3    183
 2    102
 0     55
 Name: bsmtfin type 1, dtype: int64,
 1    248
 6    243
 5    136
 3    105
 4     69
 2     52
 0     25
 Name: bsmtfin type 1, dtype: int64)

### "bsmtfin type 2" 
According to documentation, "bsmtfin type 2" is an ordinal feature with 7 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1110]:
df_train['bsmtfin type 2'].isnull().sum()

56

In [1111]:
df_train['bsmtfin type 2'].value_counts() , df_test['bsmtfin type 2'].value_counts()

(Unf    1749
 Rec      80
 LwQ      60
 BLQ      48
 ALQ      35
 GLQ      23
 Name: bsmtfin type 2, dtype: int64,
 Unf    749
 LwQ     29
 Rec     26
 BLQ     20
 ALQ     18
 GLQ     11
 Name: bsmtfin type 2, dtype: int64)

In [1112]:
col_name = "bsmtfin type 2"
col_fix_dict = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(1    1749
 3      80
 2      60
 0      56
 4      48
 5      35
 6      23
 Name: bsmtfin type 2, dtype: int64,
 1    749
 2     29
 3     26
 0     25
 4     20
 5     18
 6     11
 Name: bsmtfin type 2, dtype: int64)

### "electrical" 
According to documentation, "electrical" is an ordinal feature with 5 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1113]:
df_train['electrical'].value_counts() , df_test['electrical'].value_counts()

(SBrkr    1868
 FuseA     140
 FuseF      35
 FuseP       7
 Mix         1
 Name: electrical, dtype: int64,
 SBrkr    813
 FuseA     48
 FuseF     15
 FuseP      1
 Name: electrical, dtype: int64)

In [1114]:
col_name = "electrical"
col_fix_dict = {'SBrkr':5,'FuseA':4,'FuseF':3,'FuseP':2,'Mix':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(5    1868
 4     140
 3      35
 2       7
 1       1
 Name: electrical, dtype: int64,
 5    813
 4     48
 3     15
 2      1
 0      1
 Name: electrical, dtype: int64)

### "'fence'" 
According to documentation, "'fence'" is an ordinal feature with 5 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1115]:
df_train['fence'].value_counts() , df_test['fence'].value_counts()

(MnPrv    227
 GdPrv     83
 GdWo      80
 MnWw      10
 Name: fence, dtype: int64,
 MnPrv    103
 GdPrv     35
 GdWo      32
 MnWw       2
 Name: fence, dtype: int64)

In [1116]:
col_name = "fence"
col_fix_dict = {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'NA':0}        # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(0    1651
 3     227
 4      83
 2      80
 1      10
 Name: fence, dtype: int64,
 0    706
 3    103
 4     35
 2     32
 1      2
 Name: fence, dtype: int64)

### "fireplace qu" 
According to documentation, "fireplace qu" is an ordinal feature with 6 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1117]:
df_train['fireplace qu'].value_counts() , df_test['fireplace qu'].value_counts()

(Gd    523
 TA    407
 Fa     59
 Po     31
 Ex     31
 Name: fireplace qu, dtype: int64,
 Gd    220
 TA    193
 Fa     16
 Po     15
 Ex     12
 Name: fireplace qu, dtype: int64)

In [1118]:
col_name = "fireplace qu"
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(0    1000
 4     523
 3     407
 2      59
 1      31
 5      31
 Name: fireplace qu, dtype: int64,
 0    422
 4    220
 3    193
 2     16
 1     15
 5     12
 Name: fireplace qu, dtype: int64)

### "garage area" 
According to documentation, "garage area" is a continuous feature representing finished square feet. We expect many unique values. We will fix the null values and convert the object values to numeric.

In [1119]:
df_train['garage area'].isnull().sum() , df_test['garage area'].isnull().sum()

(1, 0)

In [1120]:
col_name = "garage area"
col_fix_dict = {'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].isnull().sum(), df_test[col_name].isnull().sum()    # Double check results

(0, 0)

### "garage cars" 
According to documentation, "garage cars" is a discrete feature is a discrete feature representing a count.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values, no need to convert the object values to numeric.

In [1121]:
df_train['garage cars'].value_counts() , df_test['garage cars'].value_counts()

(2.00    1136
 1.00     524
 3.00     263
 0.00     113
 4.00      13
 5.00       1
 Name: garage cars, dtype: int64,
 2    467
 1    254
 3    110
 0     44
 4      3
 Name: garage cars, dtype: int64)

In [1122]:
col_name = "garage cars"
col_fix_dict = {'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(2.00    1136
 1.00     524
 3.00     263
 0.00     114
 4.00      13
 5.00       1
 Name: garage cars, dtype: int64,
 2    467
 1    254
 3    110
 0     44
 4      3
 Name: garage cars, dtype: int64)

### "garage cond" 
According to documentation, "garage cond" is an ordinal feature with 6 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1123]:
df_train['garage cond'].value_counts() , df_test['garage cond'].value_counts()

(TA    1868
 Fa      47
 Gd      12
 Po       8
 Ex       2
 Name: garage cond, dtype: int64,
 TA    796
 Fa     27
 Po      6
 Gd      3
 Ex      1
 Name: garage cond, dtype: int64)

In [1124]:
col_name = "garage cond"
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(3    1868
 0     114
 2      47
 4      12
 1       8
 5       2
 Name: garage cond, dtype: int64,
 3    796
 0     45
 2     27
 1      6
 4      3
 5      1
 Name: garage cond, dtype: int64)

### "garage finish" 
According to documentation, "garage finish" is an ordinal feature with 4 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1125]:
df_train['garage finish'].value_counts() , df_test['garage finish'].value_counts()

(Unf    849
 RFn    579
 Fin    509
 Name: garage finish, dtype: int64,
 Unf    382
 RFn    233
 Fin    218
 Name: garage finish, dtype: int64)

In [1126]:
col_name = "garage finish"
col_fix_dict = {'Fin':3,'RFn':2,'Unf':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(1    849
 2    579
 3    509
 0    114
 Name: garage finish, dtype: int64,
 1    382
 2    233
 3    218
 0     45
 Name: garage finish, dtype: int64)

### "garage qual" 
According to documentation, "garage qual" is an ordinal feature with 6 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1127]:
df_train['garage qual'].value_counts() , df_test['garage qual'].value_counts()

(TA    1832
 Fa      82
 Gd      18
 Ex       3
 Po       2
 Name: garage qual, dtype: int64,
 TA    782
 Fa     42
 Gd      6
 Po      3
 Name: garage qual, dtype: int64)

In [1128]:
col_name = "garage qual"
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(3    1832
 0     114
 2      82
 4      18
 5       3
 1       2
 Name: garage qual, dtype: int64,
 3    782
 0     45
 2     42
 4      6
 1      3
 Name: garage qual, dtype: int64)

### "garage type" 
According to documentation, "garage type" is an nominal feature with 7 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1129]:
df_train['garage type'].value_counts() , df_test['garage type'].value_counts()

(Attchd     1213
 Detchd      536
 BuiltIn     132
 Basment      27
 2Types       19
 CarPort      11
 Name: garage type, dtype: int64,
 Attchd     518
 Detchd     246
 BuiltIn     53
 Basment      9
 2Types       4
 CarPort      4
 Name: garage type, dtype: int64)

In [1130]:
col_name = "garage type"
col_fix_dict = {'2Types':6,'Attchd':5,'Basment':4,'BuiltIn':3,'CarPort':2,'Detchd':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(5    1213
 1     536
 3     132
 0     113
 4      27
 6      19
 2      11
 Name: garage type, dtype: int64,
 5    518
 1    246
 3     53
 0     44
 4      9
 6      4
 2      4
 Name: garage type, dtype: int64)

### "garage yr blt" 
According to documentation, "garage yr blt" is a discrete feature with many possible values.  We will fix the null values, no need to convert the object values to numeric.

In [1131]:
df_train['garage yr blt'].isnull().sum() , df_test['garage yr blt'].isnull().sum()

(114, 45)

In [1132]:
col_name = "garage yr blt"
col_fix_dict = {'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].isnull().sum() , df_test[col_name].isnull().sum()     # Double check results

(0, 0)

### "lot frontage" 
According to documentation, "lot frontage" is an continuos feature with many possible values.  We will fix the null values, no need to convert the object values to numeric.  In order to fix the values, we hypothesized that for each lot configuration or each lot shape, there was likely a mean lot frontage we could use, rather than setting all of the null values to 0.  Ultimately, after examining a number of combinations, we choose to use the mean lot frontage for the given lot configuration, rather than lot shape.  When trying to use the shape / config combination, there were a number of possible combinations missing from the train data. Given more time, these probably could have been infered with some basic algebra.

In [1133]:
df_train['lot frontage'].isnull().sum() , df_test['lot frontage'].isnull().sum()

(330, 160)

In [1134]:
# for each shape, or each config, there may be an average frontage we can use to approximate the missing values.  
# Or maybe for each shape/config combination?

df_train['lot frontage'].value_counts().head(), df_test['lot frontage'].value_counts().head()

# looks like there are all rounded ot nearest foot

(60.00    179
 70.00     96
 80.00     94
 50.00     90
 65.00     71
 Name: lot frontage, dtype: int64,
 60.00    97
 80.00    43
 70.00    37
 75.00    37
 50.00    27
 Name: lot frontage, dtype: int64)

In [1135]:
# lot config = Lot configuration
df_train['lot config'].value_counts(), df_test['lot config'].value_counts()

(Inside     1503
 Corner      348
 CulDSac     131
 FR2          60
 FR3           9
 Name: lot config, dtype: int64,
 Inside     636
 Corner     163
 CulDSac     49
 FR2         25
 FR3          5
 Name: lot config, dtype: int64)

In [1136]:
# lot shape = General shape of property
df_train['lot shape'].value_counts(), df_test['lot shape'].value_counts()

(Reg    1295
 IR1     692
 IR2      55
 IR3       9
 Name: lot shape, dtype: int64,
 Reg    564
 IR1    286
 IR2     21
 IR3      7
 Name: lot shape, dtype: int64)

In [1137]:
df_train.groupby('lot config', as_index=False)['lot frontage'].mean()
# This makes sense
# Inside = Inside lot, I would expect to have little frontage
# Corner = Corner lot, I would expect to have more frontage than an inside lot
# CulDSac = Cul-de-sac, I would expect to have less frontage than an inside lot
# FR2 = Frontage on 2 sides of property , I would expect to have more frontage than an inside lot
# FR3 = Frontage on 3 sides of property , I would expect to have more frontage than an FR2

,lot config,lot frontage
0,Corner,83.25
1,CulDSac,55.23
2,FR2,60.84
3,FR3,87.00
4,Inside,66.95


In [1138]:
df_train.groupby('lot shape', as_index=False)['lot frontage'].mean()
# Not sure this is as useful
# Reg = Regular
# IR1 = Slightly irregular
# IR2 = Moderately Irregular
# IR3 = Irregular

,lot shape,lot frontage
0,IR1,74.58
1,IR2,59.42
2,IR3,115.67
3,Reg,66.97


In [1139]:
df_train[df_train['lot config']=='Corner'].groupby('lot shape', as_index=False)['lot frontage'].mean()

,lot shape,lot frontage
0,IR1,93.30
1,IR2,82.00
2,IR3,224.00
3,Reg,76.94


In [1140]:
df_train[df_train['lot config']=='Corner'].groupby('lot shape', as_index=False)['lot frontage'].count() 
# There arent many IR2 or IR3 corner lots, so taking the mean combination of config and shape probably doesnt make sense.

,lot shape,lot frontage
0,IR1,90
1,IR2,1
2,IR3,2
3,Reg,188


In [1141]:
df_train[df_train['lot config']=='CulDSac'].groupby('lot shape', as_index=False)['lot frontage'].mean()   
# There isnt availabel data for Cul de sac's with all shapes

,lot shape,lot frontage
0,IR1,57.26
1,IR2,49.31
2,Reg,42.00


In [1142]:
df_train[df_train['lot config']=='CulDSac'].groupby('lot shape', as_index=False)['lot frontage'].count() 
# There arent many reg cul de sac's, so taking the mean combination of config and shape probably doesnt make sense.

,lot shape,lot frontage
0,IR1,53
1,IR2,16
2,Reg,1


In [1143]:
# Not going to bother checking the other combinations.  We'll just use the mean by config

In [1144]:
# find the missing lot frontage where for each config value and replace it with the mean frontage for that config value
for val in df_train['lot config'].unique():
    df_train.loc[(df_train['lot frontage'].isnull()) & (df_train['lot config'] == val), 'lot frontage'] = df_train[df_train['lot config']==val]['lot frontage'].mean()
    
for val in df_test['lot config'].unique():
    df_test.loc[(df_test['lot frontage'].isnull()) & (df_test['lot config'] == val), 'lot frontage'] = df_test[df_test['lot config']==val]['lot frontage'].mean()    

In [1145]:
df_train['lot frontage'].isnull().sum(), df_test['lot frontage'].isnull().sum()

(0, 0)

### "mas vnr area" 
According to documentation, "mas vnr area" is a continuous feature with many possible values.  That lines up with what we see in the data.  We will fix the null values, no need to convert the object values to numeric.

In [1146]:
df_train['mas vnr area'].isnull().sum() , df_test['mas vnr area'].isnull().sum()

(22, 1)

In [1147]:
col_name = "mas vnr area"
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].isnull().sum() , df_test[col_name].isnull().sum()     # Double check results

(0, 0)

### "mas vnr type" 
According to documentation, "mas vnr type" is an nominal feature with 5 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1148]:
df_train['mas vnr type'].value_counts() , df_test['mas vnr type'].value_counts()

(None       1218
 BrkFace     630
 Stone       168
 BrkCmn       13
 Name: mas vnr type, dtype: int64,
 None       534
 BrkFace    250
 Stone       80
 BrkCmn      12
 CBlock       1
 Name: mas vnr type, dtype: int64)

In [1149]:
col_name = "mas vnr type"
col_fix_dict = {'BrkCmn':4,'BrkFace':3,'CBlock':2,'Stone':1,'None':0, 'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(0    1240
 3     630
 1     168
 4      13
 Name: mas vnr type, dtype: int64,
 0    535
 3    250
 1     80
 4     12
 2      1
 Name: mas vnr type, dtype: int64)

### "misc feature" 
According to documentation, "misc feature" is an nominal feature with 6 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1150]:
df_train['misc feature'].value_counts() , df_test['misc feature'].value_counts()

(Shed    56
 Gar2     4
 Othr     3
 TenC     1
 Elev     1
 Name: misc feature, dtype: int64,
 Shed    39
 Othr     1
 Gar2     1
 Name: misc feature, dtype: int64)

In [1151]:
col_name = "misc feature"
col_fix_dict = {'Elev':5,'Gar2':4,'Othr':3,'Shed':2,'TenC':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(0    1986
 2      56
 4       4
 3       3
 1       1
 5       1
 Name: misc feature, dtype: int64,
 0    837
 2     39
 3      1
 4      1
 Name: misc feature, dtype: int64)

### "pool qc" 
According to documentation, "pool qc" is an ordinal feature with 5 possible values.  That lines up with what we see in the data, though some values are missing from the Test data.  That's ok. We will fix the null values and convert the object values to numeric.

In [1152]:
df_train['pool qc'].value_counts() , df_test['pool qc'].value_counts()

(Gd    4
 Fa    2
 TA    2
 Ex    1
 Name: pool qc, dtype: int64,
 Ex    3
 TA    1
 Name: pool qc, dtype: int64)

In [1153]:
col_name = "pool qc"
col_fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].value_counts() , df_test[col_name].value_counts()     # Double check results

(0    2042
 4       4
 2       2
 3       2
 5       1
 Name: pool qc, dtype: int64,
 0    874
 5      3
 3      1
 Name: pool qc, dtype: int64)

### "total bsmt sf" 
According to documentation, "bsmt cond" is a continuous feature with many possible values.  That lines up with what we see in the data.  We will fix the null values, no need to convert the object values to numeric.

In [1154]:
df_train['total bsmt sf'].isnull().sum() , df_test['total bsmt sf'].isnull().sum()

(1, 0)

In [1155]:
col_name = "total bsmt sf"
col_fix_dict = {'NA':0}       # create a dictionary of the values in the data and the numeric values we want them to have

fill_nulls_convert(col_name, col_fix_dict)       # Pase the column name and dictionary of values to the fix function

df_train[col_name].isnull().sum() , df_test[col_name].isnull().sum()     # Double check results

(0, 0)

## Double Check

We now that we have filled in any missing values, and conversted some of our object (string) columns to numeric along the way, let's make sure we didn't miss any null values.


In [1156]:
for col in df_train:
    x = df_train[col].isnull().sum()
    if x > 0: 
        print(f'df_train{[col]} : {x} null vals.')
    if col != 'saleprice':   # Skip 'saleprice' as it is missing from the test data
        y = df_test[col].isnull().sum() 
        if y > 0: 
            print(f'df_test{[col]} : {y} null vals.')
            
# It's worth noting that the first time we ran this block, 
# there was 1 feature that we missed above.  
# Good thing we included the double check....

## Given more time...

Similar to the way we estimated lot frontage, some of the missing values that we just assumed to be 0 could probably be estimated as well.  Given more time, we would like to try to estimating the values below based on the logic included.

In [1157]:
# df_train['bsmt cond'] : An estimate given a combination of Bsmt Quality, Bsmt Exposure, Bsmt Type, , neighborhood, and Bsmt Sq Ft
    
# df_train['bsmt exposure'] : An estimate given a combination of Bsmt Quality, Bsmt Type, Lot Config, Ext QUal and Land Slope

# df_train['bsmt full bath'] : An estimate given a combination of Bsmt Quality, Bsmt Exposure, Bsmt Type, BsmtFin Type 1, and Bsmt Sq Ft

# df_train['bsmt half bath'] : An estimate given a combination of Bsmt Quality, Bsmt Exposure, Bsmt Type, BsmtFin Type 1, and Bsmt Sq Ft

# df_train['bsmt qual'] : An estimate given a combination of all other bsmt features, overall cond, overall qual

# df_train['bsmt unf sf'] : An estimate given a combination of all other bsmt features, overall cond, overall qual

# df_train['bsmtfin sf 1'] : An estimate given a combination of all other bsmt features, overall cond, overall qual

# df_train['bsmtfin type 1'] : An estimate given a combination of all other bsmt features, overall cond, overall qual

# df_test['electrical'] : An estimate based onthe combination of Yr Built, Yr Remod, Neighborhood, Overall Qual, OVerall Cond, Heating, and Central Air

# df_train['fence'] An estimate based on a combination of Overal Qual, Ext features, Pool QC, neighborhood

# df_train['garage area'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood.

# df_train['garage cars'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood..

# df_train['garage cond'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood..

# df_train['garage finish'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood..

# df_train['garage qual'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood.

# df_train['garage type'] : An estimate based on all other Garage features, overall cond, overall qual, exter 1, house style, neighborhood..

# df_train['garage yr blt'] : An estimate based on all other Garage features, year built, year remodeled

# df_train['mas vnr type'] : An estimate based on overall cond, overall qual, exter 1, house style, neighborhood

# df_train['total bsmt sf'] :  An estimate given a combination of all other bsmt features, overall cond, overall qual, neighborhood

## <span style='background :#3FBFBF' >*******************  Non-Numeic Clean Up *******************</span>  

In [1158]:
# Looking at the documentation, we can binarize 'Street' and 'Central Air' 

In [1159]:
df_train['street'].value_counts(), df_test['street'].value_counts()

(Pave    2044
 Grvl       7
 Name: street, dtype: int64,
 Pave    873
 Grvl      5
 Name: street, dtype: int64)

In [1160]:
df_train['street'] = df_train['street'].map({'Pave':1, 'Grvl':0})
df_test['street'] = df_test['street'].map({'Pave':1, 'Grvl':0})

df_train['street'].head(), df_test['street'].head()

(0    1
 1    1
 2    1
 3    1
 4    1
 Name: street, dtype: int64,
 0    1
 1    1
 2    1
 3    1
 4    1
 Name: street, dtype: int64)

In [1161]:
df_train['central air'].value_counts(), df_test['central air'].value_counts()

(Y    1910
 N     141
 Name: central air, dtype: int64,
 Y    823
 N     55
 Name: central air, dtype: int64)

In [1162]:
df_train['central air'] = df_train['central air'].map({'Y':1, 'N':0})
df_test['central air'] = df_test['central air'].map({'Y':1, 'N':0})
df_train['central air'].value_counts(), df_test['central air'].value_counts()

(1    1910
 0     141
 Name: central air, dtype: int64,
 1    823
 0     55
 Name: central air, dtype: int64)

In [1163]:
obj_cols = df_train.dtypes[df_train.dtypes == 'object'].index
obj_cols

Index(['bldg type', 'condition 1', 'condition 2', 'exter cond', 'exter qual',
       'exterior 1st', 'exterior 2nd', 'foundation', 'functional', 'heating',
       'heating qc', 'house style', 'kitchen qual', 'land contour',
       'land slope', 'lot config', 'lot shape', 'ms zoning', 'neighborhood',
       'paved drive', 'roof matl', 'roof style', 'sale type', 'utilities'],
      dtype='object')

In [1164]:
# accoring to documentation, these columns are ordinal. So we should be able to assign meaningful 
# values where they aren't already numeric

# Exter Cond, Exter Qual, Land Slope, Lot Shape, Utilities,  
# Bsmt Cond, Bsmt Exposure, BsmtFin Type 1,BsmtFinType 2, HeatingQC, Electrical, KitchenQual, Functional, FireplaceQu, 
# Garage Finish, Garage Qual, Garage Cond, Paved Drive, Pool QC, Fence 

# it looks like Fireplace Qu, Garage Qual, Garage Cond,  Pool QC, Fence all use the same scale

# so do BsmtFin Type 1, and BsmtFinType 2

# so do exter qual, exter cond, kitchen qual, heating qc

# so do Bsmt Qual, Bsmt Cond


In [1165]:
# Only 5 different values
df_train=pd.get_dummies(df_train, columns=['bldg type'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['bldg type'], drop_first=False)

In [1166]:
# Some of these are listed as positive specifically, and normal, but these others dont sounds positive.
# After some googling, it looks like there are definitely positive, negative, normal conditions.  
# It looks like the recomendation is to score them as 2 for postive, 1 for normal, and 0 for negative
# then add the score for each property and use the combined score in the model.

     # 0  Artery	Adjacent to arterial street
     # 0  Feedr	Adjacent to feeder street	
     # 1  Norm	Normal	
     # 0  RRNn	Within 200' of North-South Railroad
     # 0  RRAn	Adjacent to North-South Railroad
     # 2  PosN	Near positive off-site feature--park, greenbelt, etc.
     # 2  PosA	Adjacent to postive off-site feature
     # 0  RRNe	Within 200' of East-West Railroad
     # 0  RRAe	Adjacent to East-West Railroad

In [1167]:
fix_dict_conds = {'Artery':0,
                  'Feedr':0,
                  'Norm':1,
                  'RRNn':0,
                  'RRAn':0,
                  'PosN':2,
                  'PosA':2,
                  'RRNe':0,
                  'RRAe':0   } # create a dictionary of the values in the data and the numeric values you want them to have
df_train['condition 1'] = df_train['condition 1'].replace(fix_dict_conds)
df_test['condition 1'] = df_test['condition 1'].replace(fix_dict_conds)
df_train['condition 2'] = df_train['condition 2'].replace(fix_dict_conds)
df_test['condition 2'] = df_test['condition 2'].replace(fix_dict_conds)

df_train['conditions'] = df_train['condition 1'] + df_train['condition 2']
df_test['conditions'] = df_test['condition 1'] + df_test['condition 2']

In [1168]:
fix_dict = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1} # create a dictionary of the values in the data and the numeric values you want them to have
cols_to_fix = ['exter qual','exter cond','kitchen qual','heating qc']  # create a list of the columns to fix
for col in cols_to_fix:
    df_train[col] = df_train[col].replace(fix_dict)
    df_test[col] = df_test[col].replace(fix_dict)


In [1169]:
# The documentation says that the exterior 1 and exterior 2 data are nominal
# but there are some large differences in mean sale price of ext 1

In [1170]:
df_train.groupby('exterior 1st')['saleprice'].agg(['mean', 'count']).sort_values(by = 'mean')

,mean,count
exterior 1st,,
AsphShn,82375.00,1
CBlock,93250.00,2
AsbShng,103183.33,33
BrkComm,131000.00,3
Stucco,139712.96,27
Wd Sdng,142940.89,276
MetalSd,154402.05,331
WdShing,155616.82,45
HdBoard,159582.47,300


In [1171]:
df_train.groupby('exterior 2nd')['saleprice'].agg(['mean', 'count']).sort_values(by = 'mean')

,mean,count
exterior 2nd,,
CBlock,92000.00,2
AsbShng,107108.93,28
AsphShn,132125.00,3
Brk Cmn,139958.82,17
Stone,145420.67,6
Wd Sdng,145654.40,262
Stucco,147725.33,30
Wd Shng,154244.44,63
MetalSd,155978.64,324


In [1172]:
## some of the values from Ext 1 and Ext 2 dont' quite match on the train data, checking the test data next
print(df_train['exterior 1st'].unique())
print(df_train['exterior 2nd'].unique())
print(type(df_train['exterior 1st'].unique()))

print(list(set(df_train['exterior 1st']) - set(df_train['exterior 2nd'])))
print(list(set(df_train['exterior 2nd']) - set(df_train['exterior 1st'])))

['HdBoard' 'VinylSd' 'Wd Sdng' 'BrkFace' 'Plywood' 'MetalSd' 'AsbShng'
 'CemntBd' 'WdShing' 'Stucco' 'BrkComm' 'Stone' 'CBlock' 'ImStucc'
 'AsphShn']
['Plywood' 'VinylSd' 'Wd Sdng' 'HdBoard' 'MetalSd' 'AsbShng' 'CmentBd'
 'Wd Shng' 'BrkFace' 'Stucco' 'Brk Cmn' 'ImStucc' 'Stone' 'CBlock'
 'AsphShn']
<class 'numpy.ndarray'>
['CemntBd', 'WdShing', 'BrkComm']
['Brk Cmn', 'Wd Shng', 'CmentBd']


In [1173]:
## alot of mis-match on the test data
print(df_test['exterior 1st'].unique())
print(df_test['exterior 2nd'].unique())


print(list(set(df_test['exterior 1st']) - set(df_test['exterior 2nd'])))
print(list(set(df_test['exterior 2nd']) - set(df_test['exterior 1st'])))

['AsbShng' 'Plywood' 'VinylSd' 'Wd Sdng' 'CemntBd' 'MetalSd' 'HdBoard'
 'BrkComm' 'Stucco' 'WdShing' 'BrkFace' 'PreCast' 'AsphShn']
['AsbShng' 'Plywood' 'VinylSd' 'Wd Sdng' 'CmentBd' 'MetalSd' 'BrkFace'
 'Stucco' 'HdBoard' 'Wd Shng' 'ImStucc' 'Brk Cmn' 'PreCast' 'CBlock'
 'AsphShn' 'Other']
['CemntBd', 'WdShing', 'BrkComm']
['Other', 'ImStucc', 'Brk Cmn', 'CmentBd', 'CBlock', 'Wd Shng']


In [1174]:
# make Ext 2nd values match those of Ext 1st
fix_dict_ext2 = {'Brk Cmn':'BrkComm',
                  'Wd Shng':'WdShing',
                  'CmentBd':'CemntBd'  } # create a dictionary of the values in the data and the numeric values you want them to have
df_train['exterior 2nd'] = df_train['exterior 2nd'].replace(fix_dict_ext2)
df_test['exterior 2nd'] = df_test['exterior 2nd'].replace(fix_dict_ext2)

In [1175]:
# Lets see if the mean sale price of Ext 1 in train  is similar to that of Ext 2

In [1176]:
df_train.groupby('exterior 1st')['saleprice'].agg(['mean', 'count']).sort_values(by = 'mean')

,mean,count
exterior 1st,,
AsphShn,82375.00,1
CBlock,93250.00,2
AsbShng,103183.33,33
BrkComm,131000.00,3
Stucco,139712.96,27
Wd Sdng,142940.89,276
MetalSd,154402.05,331
WdShing,155616.82,45
HdBoard,159582.47,300


In [1177]:
df_train.groupby('exterior 2nd')['saleprice'].agg(['mean', 'count']).sort_values(by = 'mean')

,mean,count
exterior 2nd,,
CBlock,92000.00,2
AsbShng,107108.93,28
AsphShn,132125.00,3
BrkComm,139958.82,17
Stone,145420.67,6
Wd Sdng,145654.40,262
Stucco,147725.33,30
WdShing,154244.44,63
MetalSd,155978.64,324


In [1178]:
# the mean sale price of Ext 1 and Ext 2 are similar but there are a few big differences.  
# I'm going to assign each it's own Ordinal value, then, similar to what we did with 
# conditions, combine the two columns

In [1179]:
    #    AsbShng	Asbestos Shingles
    #    AsphShn	Asphalt Shingles
    #    BrkComm	Brick Common
    #    BrkFace	Brick Face
    #    CBlock	Cinder Block
    #    CemntBd	Cement Board
    #    HdBoard	Hard Board
    #    ImStucc	Imitation Stucco
    #    MetalSd	Metal Siding
    #    Other	Other
    #    Plywood	Plywood
    #    PreCast	PreCast	
    #    Stone	Stone
    #    Stucco	Stucco
    #    VinylSd	Vinyl Siding
    #    Wd Sdng	Wood Siding
    #    WdShing	Wood Shingles

In [1180]:
fix_dict_ext1 = {
    'AsphShn':1,
    'CBlock':2,
    'AsbShng':3,
    'BrkComm':4,
    'Stucco':5,
    'Wd Sdng':6,
    'MetalSd':7,
    'PreCast':8,
    'WdShing':9,
    'HdBoard':10,
    'Plywood':11,
    'BrkFace':12,
    'VinylSd':13,
    'CemntBd':14,
    'Stone':15,
    'ImStucc':16  } # create a dictionary of the values in the data and the numeric values you want them to have
df_train['exterior 1st'] = df_train['exterior 1st'].replace(fix_dict_ext1)
df_test['exterior 1st'] = df_test['exterior 1st'].replace(fix_dict_ext1)


In [1181]:
print(df_test['exterior 1st'].unique())
print(df_test['exterior 1st'].value_counts())
# Not sure what to do about PreCast, maybe just assume the mean sale price? stick it in the middle?
#df_test['saleprice'].mean()

[ 3 11 13  6 14  7 10  4  5  9 12  8  1]
13    302
6     144
10    142
7     119
11     69
14     35
12     24
5      16
3      11
9      11
4       3
8       1
1       1
Name: exterior 1st, dtype: int64


In [1182]:

fix_dict_ext2 = {
    'Other':0,
    'CBlock':1,
    'AsbShng':2,
    'AsphShn':3,
    'BrkComm':4,
    'Stone':5,
    'Wd Sdng':6,
    'Stucco':7,
    'PreCast':8,
    'WdShing':9,
    'MetalSd':10,
    'HdBoard':11,
    'Plywood':12,
    'BrkFace':13,
    'VinylSd':14,
    'ImStucc':15,
    'CemntBd':16} # create a dictionary of the values in the data and the numeric values you want them to have


df_train['exterior 2nd'] = df_train['exterior 2nd'].replace(fix_dict_ext2)
df_test['exterior 2nd'] = df_test['exterior 2nd'].replace(fix_dict_ext2)


In [1183]:
print(df_test['exterior 2nd'].unique())
print(df_test['exterior 2nd'].value_counts())
# Not sure what to do about PreCast, maybe just assume the mean sale price? stick it in the middle?


[ 2 12 14  6 16 10 13  7 11  9 15  4  8  1  3  0]
14    294
6     135
11    131
10    123
12     89
16     35
9      18
7      17
13     13
2      10
4       5
15      4
8       1
1       1
3       1
0       1
Name: exterior 2nd, dtype: int64


In [1184]:

df_train['exterior 2nd'] = df_train['exterior 2nd'].replace(fix_dict_ext2)
df_test['exterior 2nd'] = df_test['exterior 2nd'].replace(fix_dict_ext2)

df_train['exteriors'] = df_train['exterior 1st'] + df_train['exterior 2nd']
df_test['exteriors'] = df_test['exterior 1st'] + df_test['exterior 2nd']

In [1185]:
df_train=pd.get_dummies(df_train, columns=['foundation'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['foundation'], drop_first=False)

In [1186]:
fix_dict_big= {
#functional
    'Typ':8,
    'Min1':7,
    'Min2':6,
    'Mod':5,
    'Maj1':4,
    'Maj2':3,
    'Sev':2,
    'Sal':1,
#garage finish    
    'Fin':3,
    'RFn':2,
    'Unf':1,
    'NA':0,
#land slope
    'Gtl':3,
    'Mod':2,
    'Sev':1,    
#lot shape
    'Reg':4,
    'IR1':3,
    'IR2':2,
    'IR3':1,
#paved drive
    'Y' : 2,
    'P' : 1,
    'N' : 0,    
#utilities
    'AllPub':4,
    'NoSewr':3,
    'NoSeWa':2,
    'ELO':1,   
}

In [1187]:
cols_to_fix_big = ['functional','garage finish','lot shape','land slope','paved drive','utilities']  # create a list of the columns to fix

In [1188]:
for col in cols_to_fix_big:
    df_train[col] = df_train[col].replace(fix_dict_big)
    df_test[col] = df_test[col].replace(fix_dict_big)

In [ ]:
df_train=pd.get_dummies(df_train, columns=['alley'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['allley'], drop_first=False)

In [ ]:
df_train=pd.get_dummies(df_train, columns=['house style'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['house style'], drop_first=False)

In [1190]:
df_train=pd.get_dummies(df_train, columns=['land contour'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['land contour'], drop_first=False)

In [1191]:
df_train=pd.get_dummies(df_train, columns=['lot config'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['lot config'], drop_first=False)

In [1192]:
df_train=pd.get_dummies(df_train, columns=['ms zoning'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['ms zoning'], drop_first=False)

In [1193]:
df_train=pd.get_dummies(df_train, columns=['neighborhood'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['neighborhood'], drop_first=False)

In [1194]:
df_train=pd.get_dummies(df_train, columns=['roof matl'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['roof matl'], drop_first=False)

In [1195]:
df_train=pd.get_dummies(df_train, columns=['roof style'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['roof style'], drop_first=False)

In [1196]:
df_train=pd.get_dummies(df_train, columns=['sale type'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['sale type'], drop_first=False)

In [1197]:
df_train=pd.get_dummies(df_train, columns=['heating'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['heating'], drop_first=False)

In [1199]:
df_train=pd.get_dummies(df_train, columns=['garage type'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['garage type'], drop_first=False)

In [1200]:
df_train=pd.get_dummies(df_train, columns=['mas vnr type'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['mas vnr type'], drop_first=False)

In [1201]:
df_train=pd.get_dummies(df_train, columns=['misc feature'], drop_first=False)
df_test=pd.get_dummies(df_test, columns=['misc feature'], drop_first=False)

In [1202]:
obj_cols = df_train.dtypes[df_train.dtypes == 'object'].index
obj_cols

Index([], dtype='object')

In [1203]:
print('Finished')

Finished


In [1204]:
################################################################################

In [1205]:
################################################################################

In [1207]:
df_train.columns = df_train.columns.str.lower() # column names to lower case
df_test.columns = df_test.columns.str.lower() # column names to lower case

In [1208]:
df_train.to_csv('./datasets/cleaned_train4.csv', index=False)

In [1209]:
df_test.to_csv('./datasets/cleaned_test4.csv', index=False)

In [ ]:
fill NA cleaning

In [ ]:
dummies One Hot Encoding = Feature ENgineering